In [1]:
from PIL import  Image
import Data_augmentation
import Normalizer_SVD
import os
import numpy as np
import time

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
def norm_transform(img, overlap = False):
    #normalize input image
    img_normalized, _, _ = Normalizer_SVD.normalize_stain(img)
        
    #patch input image
    patches,_,_ = Data_augmentation.py_patch(img_normalized, overlap = overlap)

    patches_transformed = []
    # apply trasformations(mirror, rotation) to all patches
    for patch in patches:
        transformed_patch = Data_augmentation.transform(patch)
        patches_transformed += transformed_patch
            
    return patches_transformed
    

In [3]:

def preprocess_train(target,path_in, path_save,
               disp = False, data_type = 'Train'):

    data = list(os.walk(path_in+"/"+target))[0][-1]
    list_images =np.array([np.array(Image.open(img)) for img in data])
        
    for i,img in enumerate(list_images):
        
        patches_transformed = norm_transform(img, overlap = True)
            
        for count, patch in enumerate(patches_transformed):
            
            img_transformed = Image.fromarray(patch,'RGB')
            img_transformed.save(path_save + data_type+'/'+target+'/t'+str(i)+'_'+str(count+1)+'.tif')
            
        if disp == True:
            Data_augmentation.show_images(patches_transformed)
    
    target_list = [target for i in range(len(patches_transformed))]
  
    return patches_transformed, target_list
   

In [4]:
def preprocess_test(path_in, path_save):
    
    # dict: {img_name: ..., target: ...., set_type:...}
    # set -> initial or extended
   
    file_labels = open(path_save + '/labels.txt','w')
    
    with open(path_in +'/labels.txt') as f:
        
        for line in f:
            img_name, target, set_type = line.split()
            
            img = np.array(Image.open(path_in+'/'+img_name+'.tif'))
            
            patches_transformed = norm_transform(img, overlap = False)
            
            img_labels = []
            for count, patch in enumerate(patches_transformed):
            
                img_transformed = Image.fromarray(patch,'RGB')
                img_transformed.save(path_save + '/t'+img_name+'_'+str(count+1)+'.tif')
                
                file_labels.write(img_name+'_'+str(count+1)+'.tif '+target+' '+set_type) 
                file_labels.write('\n')
            
    
        file_labels.close()


In [1]:
#start = time.time()
#preprocess_train(...)
#preproces_test(...)
#end = time.time()

#print(f'Execution time = {end - start}')